In [1]:
library(nichenetr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2      ✔ purrr   0.3.5 
✔ tibble  3.2.1      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
## We load the networks generated in the ParameterOptimization script
lr_network_Omnipath = readRDS("../../../../../results/method_comparison/build_customDB/NicheNet/lig_rec_sources.rds")
sig_network_Omnipath = readRDS("../../../../../results/method_comparison/build_customDB/NicheNet/sig_Network.rds")
gr_network_Omnipath = readRDS("../../../../../results/method_comparison/build_customDB/NicheNet/gr_Network.rds")

# Construct NicheNet’s ligand-target model from unoptimized data source weights


In [4]:
## The interactions are weighted only based in the number of data sources that
## report them
All_sources <- unique(c(lr_network_Omnipath$source,
    sig_network_Omnipath$source, gr_network_Omnipath$source))

In [12]:
All_sources

[1] "AlzPathway"                  "Baccin2019"                 
  [3] "CellTalkDB"                  "EMBRACE"                    
  [5] "Fantom5_LRdb"                "HPMR_LRdb"                  
  [7] "HPMR_talklr"                 "HPRD"                       
  [9] "HPRD_LRdb"                   "HPRD_talklr"                
 [11] "LRdb"                        "Ramilowski2015"             
 [13] "Ramilowski2015_Baccin2019"   "STRING_talklr"              
 [15] "Wang"                        "connectomeDB2020"           
 [17] "iTALK"                       "talklr"                     
 [19] "Reactome_LRdb"               "Kirouac2010"                
 [21] "CellCall"                    "CellChatDB"                 
 [23] "CellPhoneDB"                 "Cellinker"                  
 [25] "DIP"                         "Guide2Pharma"               
 [27] "Guide2Pharma_LRdb"           "Guide2Pharma_talklr"        
 [29] "HPMR"                        "HPMR_Cellinker"             
 [31] "I2D_CellPhoneDB"             "IMEx_CellPhoneDB"           
 [33] "SIGNOR"                      "scConnect"                  
 [35] "InnateDB-All_CellPhoneDB"    "MINT_CellPhoneDB"           
 [37] "Guide2Pharma_Cellinker"      "HuRI"                       
 [39] "IntAct_CellPhoneDB"          "Guide2Pharma_CellPhoneDB"   
 [41] "DOMINO"                      "Lit-BM-17"                  
 [43] "SignaLink3"                  "BioGRID"                    
 [45] "IntAct"                      "DLRP_talklr"                
 [47] "UniProt_LRdb"                "ProtMapper"                 
 [49] "Sparser_ProtMapper"          "DLRP_Cellinker"             
 [51] "KEGG-MEDICUS"                "ICELLNET"                   
 [53] "CancerCellMap"               "NetPath"                    
 [55] "InnateDB"                    "CA1"                        
 [57] "REACH_ProtMapper"            "MatrixDB"                   
 [59] "InnateDB_SignaLink3"         "Wojtowicz2020"              
 [61] "CellPhoneDB_Cellinker"       "Adhesome"                   
 [63] "LMPID"                       "SPIKE"                      
 [65] "ACSN_SignaLink3"             "ELM"                        
 [67] "BEL-Large-Corpus_ProtMapper" "ACSN"                       
 [69] "Macrophage"                  "PhosphoPoint"               
 [71] "Reactome_SignaLink3"         "MPPI"                       
 [73] "InnateDB_CellPhoneDB"        "UniProt_CellPhoneDB"        
 [75] "CellChatDB-cofactors"        "HPRD-phos"                  
 [77] "DEPOD"                       "SIGNOR_ProtMapper"          
 [79] "TRIP"                        "RLIMS-P_ProtMapper"         
 [81] "HPRD_KEA"                    "HPRD_MIMP"                  
 [83] "KEA"                         "MIMP"                       
 [85] "PhosphoNetworks"             "PhosphoSite"                
 [87] "PhosphoSite_KEA"             "PhosphoSite_MIMP"           
 [89] "PhosphoSite_ProtMapper"      "PhosphoSite_noref"          
 [91] "iPTMnet"                     "phosphoELM"                 
 [93] "phosphoELM_KEA"              "phosphoELM_MIMP"            
 [95] "Li2012"                      "NCI-PID_ProtMapper"         
 [97] "Reactome_ProtMapper"         "MatrixDB_CellPhoneDB"       
 [99] "NetworKIN_KEA"               "dbPTM"                      
[101] "Kinexus_KEA"                 "ARN"                        
[103] "TCRcuration_SignaLink3"      "DeathDomain"                
[105] "NRF2ome"                     "ARACNe-GTEx_DoRothEA"       
[107] "ReMap_DoRothEA"              "HOCOMOCO_DoRothEA"          
[109] "DoRothEA"                    "HTRIdb"                     
[111] "HTRIdb_DoRothEA"             "TRRUST_DoRothEA"            
[113] "FANTOM4_DoRothEA"            "DoRothEA-reviews_DoRothEA"  
[115] "IntAct_DoRothEA"             "RegNetwork_DoRothEA"        
[117] "TRED_DoRothEA"               "PAZAR_DoRothEA"             
[119] "JASPAR_DoRothEA"             "TFactS_DoRothEA"            
[121] "TFe_DoRothEA"                "PAZAR

In [5]:
my_source_weights_df <- 
     tibble(source = All_sources, weight = rep(1,length(All_sources)))

In [6]:
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath, 
    source_weights_df = my_source_weights_df)

In [7]:
## We read the results of the optimization 
resultsOptimization <- readRDS("../../../../../results/method_comparison/build_customDB/NicheNet/Optimization_results.rds")

optimized_parameters = resultsOptimization %>% 
    process_mlrmbo_nichenet_optimization(
        source_names = my_source_weights_df$source %>% unique())

In [11]:
weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)
saveRDS(weighted_networks,"../../../../../results/method_comparison/build_customDB/NicheNet/weighted_networksNonSourceWeights.rds")

In [12]:
#We now generate the matrix containing the ligand-target regulatory potential scores based on the 
#weighted integrated networks.


ligands <- as.list(unique(lr_network_Omnipath$from))

ligand_target_matrix <- construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"../../../../../results/method_comparison/build_customDB/NicheNet/ligand_target_matrixNoweights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


In [13]:
## Here we also take into account the optimized source weights
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath,
    source_weights_df = optimized_parameters$source_weight_df)

weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)

ligand_target_matrix = construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"../../../../../results/method_comparison/build_customDB/NicheNet/ligand_target_matrixWithweights.rds")
saveRDS(weighted_networks,"../../../../../results/method_comparison/build_customDB/NicheNet/weighted_networksWithSourceWeights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


In [12]:
lr_network_Omnipath$pair = paste(lr_network_Omnipath$from, lr_network_Omnipath$to, sep="_")